In [1]:
from pandas.tseries.offsets import DateOffset
import ee
import yaml
import time
import os
import json
import pandas as pd
from utils import exportImageToGDrive,exportImageToGCS
from download_sen12 import *

ee.Initialize()

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [2]:
config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][0]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)

project-canopy-temp-2


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:466: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [3]:
sensor['bands']

['TCI_R', 'TCI_G', 'TCI_B']

In [4]:
source

{'name': ['full_basin_export_v10_3_months'],
 'start_date': '2019-01-01',
 'end_date': '2020-12-31',
 'geometry': 'point',
 'size': 20000,
 'resolution': 10,
 'sort_by': 'id',
 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
 'sensors': [0, 1, 2]}

In [3]:
def import_aois(csv_loc, Full_Congo_Pull=False, start_date=None,
                end_date=None, days_duration=90, poly_start=0, poly_limit=None):
    features = []
    polygons = []
    day_offset = days_duration / 2
    start_end_list = []
    
    if Full_Congo_Pull:
        with open(csv_loc,"r",encoding='utf-8') as jsonfile:
            data = json.load(jsonfile)
            for feature_id, geometry in enumerate(data["features"], 1):
                polygon = geometry["geometry"]["coordinates"][0][0]
                poly_obj = ee.Geometry.Polygon(polygon)
                feature = ee.Feature(poly_obj, {"name": feature_id})
                features.append(feature)
                
            original_date = pd.to_datetime('01-01-2020')
            start = (original_date + DateOffset(days=-30))
            end = (original_date + DateOffset(days=30))
            start_date = str(start)[:10]
            end_date = str(end)[:10]
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': 30,
                'area': 1
            }
            
            start_end_list = [date_dict] * len(features)
            
            return ee.FeatureCollection(features), start_end_list
            
    else:
        feature_id = poly_start
        
        if poly_limit:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1), nrows=poly_limit)
        else:
            df_labels = pd.read_csv(csv_loc, skiprows=range(1, poly_start+1))

        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined","tile date","area (km2)"]]
        df_labels["tile date"] = pd.to_datetime(df_labels["tile date"])
        start = (df_labels["tile date"] + DateOffset(days=-day_offset))
        end = (df_labels["tile date"] + DateOffset(days=day_offset))
        for i in range(len(start)):
            start_date = str(start[i])[:10]
            end_date = str(end[i])[:10]
            original_date = df_labels.loc[i, 'tile date']
            area = df_labels.loc[i, 'area (km2)']
            date_dict = {
                'start_date': start_date,
                'end_date': end_date,
                'original_date': original_date,
                'day_offset': day_offset,
                'area': area
            }
            start_end_list.append(date_dict)

        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])
            
        for poly in polygons:
            # create an roi. first item in Misha's label list
            feature_id += 1 
            # create geometry object, create feature object, append to features list for feature collection creation 
            polys = ee.Geometry.Polygon(poly)
            feature = ee.Feature(polys,{"name":feature_id})
            features.append(feature)
            
        return ee.FeatureCollection(features),start_end_list,df_labels


# ### CHANGE BELOW PATH ###
zhenya_label_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv"
david__label_path = 'D:/canopy_data/csvs/polygons_101320.csv'
zhenya_tilelist_path = "/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Geometry/tile_lists/shub_tile_list/tiles_polygons_full.csv"
david_grid_path = 'D:/canopy_data/Geometry/Congo_Basin_Boundary_no_islands_v5_gridded.geojson'

In [6]:
fc,date_range_list,df_labels = import_aois(david__label_path, Full_Congo_Pull=False)

In [7]:
len(date_range_list)

101

In [8]:
date_range_list[0:5]

[{'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0,
  'area': 207.84},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0,
  'area': 89.56},
 {'start_date': '2019-11-30',
  'end_date': '2020-02-28',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0,
  'area': 28.52},
 {'start_date': '2019-11-25',
  'end_date': '2020-02-23',
  'original_date': Timestamp('2020-01-09 00:00:00'),
  'day_offset': 45.0,
  'area': 7.58},
 {'start_date': '2019-11-30',
  'end_date': '2020-02-28',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0,
  'area': 169.77}]

In [9]:
drl_3_months = date_range_list.copy()

for d in drl_3_months:
    new_start = '2019-12-11'
    new_end = '2020-03-10'
    d['start_date'] = new_start
    d['end_date'] = new_end
    
drl_3_months[0:5]

[{'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0,
  'area': 207.84},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0,
  'area': 89.56},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0,
  'area': 28.52},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-09 00:00:00'),
  'day_offset': 45.0,
  'area': 7.58},
 {'start_date': '2019-12-11',
  'end_date': '2020-03-10',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0,
  'area': 169.77}]

In [10]:
len(drl_3_months)

101

In [11]:
exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict=None,
    date_range_list=drl_3_months, no_dynamic_daterage=True
)

101 features have been loaded


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [7]:
offset_dict = {
    45: 90,
    90: 180,
    180: 'two years'
}

exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict=offset_dict,
    date_range_list=date_range_list, no_dynamic_daterage=False, limit=1
)

101 features have been loaded
date range dict: {'start_date': '2019-12-11', 'end_date': '2020-03-10', 'original_date': Timestamp('2020-01-25 00:00:00'), 'day_offset': 45.0, 'area': 207.84}
Start date: 2019-12-11; End date: 2020-03-10
{'type': 'Date', 'value': 1576022400000}
collection made


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [12]:
config_dict = load_config('config.yml')
source = config_dict['data_list'][0]
sensor = config_dict['sensors'][0]
# export_folder = config_dict['drive_folder']
export_folder = config_dict['bucket']

print(export_folder)

project-canopy-temp-2


C:\Users\David\canopy\canopy-gis\google-earth-explore\GEE_DataDownloader\download_sen12.py:458: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(stream)


In [13]:
source

{'name': ['full_basin_export_v10_2_years'],
 'start_date': '2019-01-01',
 'end_date': '2020-12-31',
 'geometry': 'point',
 'size': 20000,
 'resolution': 10,
 'sort_by': 'id',
 'features_src': 'ft:19Vexm10pJcAZ8tTVbl4j0HA8w2muyPPz6-cyvdxI',
 'sensors': [0, 1, 2]}

In [14]:
drl_2_years = date_range_list.copy()

for d in drl_2_years:
    new_start = '2019-01-01'
    new_end = '2020-12-31'
    d['start_date'] = new_start
    d['end_date'] = new_end
    
drl_2_years[0:5]

[{'start_date': '2019-01-01',
  'end_date': '2020-12-31',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0,
  'area': 207.84},
 {'start_date': '2019-01-01',
  'end_date': '2020-12-31',
  'original_date': Timestamp('2020-01-25 00:00:00'),
  'day_offset': 45.0,
  'area': 89.56},
 {'start_date': '2019-01-01',
  'end_date': '2020-12-31',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0,
  'area': 28.52},
 {'start_date': '2019-01-01',
  'end_date': '2020-12-31',
  'original_date': Timestamp('2020-01-09 00:00:00'),
  'day_offset': 45.0,
  'area': 7.58},
 {'start_date': '2019-01-01',
  'end_date': '2020-12-31',
  'original_date': Timestamp('2020-01-14 00:00:00'),
  'day_offset': 45.0,
  'area': 169.77}]

In [15]:
exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict=None,
    date_range_list=drl_2_years, no_dynamic_daterage=True
)

101 features have been loaded


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [19]:
date = ee.Date(date_range_list[0]['start_date'])

date

In [20]:
date.getInfo()

{'type': 'Date', 'value': 1607644800000}

In [12]:
feature_list = fc.sort(source['sort_by']).toList(fc.size())
feature_point = ee.Feature( feature_list.get(0) )
roi = feature_point.geometry()
roi = roi.coordinates().getInfo()[0]

In [14]:
start_date = drl_3_months[0]['start_date']
end_date = drl_3_months[0]['end_date']

collection = ee.ImageCollection(sensor['name']) \
.filterDate(ee.Date(start_date), ee.Date(end_date)) \
#.filterBounds(roi)

In [15]:
collection.size().getInfo()

EEException: Collection.reduceColumns: Empty date ranges not supported for the current operation.

In [16]:
start_date

'2020-12-11'

In [17]:
end_date

'2020-03-10'

# Logging dynamic date range collection sizes

In [4]:
fc,date_range_list,df_labels = import_aois(david__label_path, Full_Congo_Pull=False)

In [5]:
offset_dict = {
    45: 90,
    90: 180,
    180: 'two years'
}

exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list=date_range_list, limit=1
)

101 features have been loaded


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [6]:
exports

([None], [])

In [7]:
exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list=date_range_list
)

101 features have been loaded


Traceback (most recent call last):
  File "C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\_ffi\loop.py", line 269, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
KeyboardInterrupt
2021-01-15T21:15:08Z


KeyboardInterrupt: 

C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [9]:
exports = process_datasource_custom_daterange(
    source, sensor, export_folder, fc, offset_dict, date_range_list=date_range_list, loop_start=81
)

101 features have been loaded


C:\Anaconda3\envs\cloud_removal\lib\site-packages\gevent\hub.py:158: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
